In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
#         self.data.sample(frac=1).reset_index(drop=True)
        self.start = 26

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_sentence = torch.tensor([self.start]+[ord(c) - ord('a') for c in self.data.iloc[idx, 0]])
#         input_sentence = self.embedding(input_sentence)
        
        target_sentence = torch.tensor([self.start]+[ord(c) - ord('a') for c in self.data.iloc[idx, 1]])
        target_sentence_encoded = torch.zeros(target_sentence.shape[0],27)
        for idx in range(target_sentence.shape[0]):
            target_sentence_encoded[idx][target_sentence[idx]] = 1.0
#         target_sentence_ = self.embedding(target_sentence)
        return (input_sentence,target_sentence,target_sentence_encoded)

In [3]:
training_data = DataLoader(CustomDataset("Data/train_data.csv"))
training_source = []
training_target = []
training_target_encoded = []
for input,target,target_encoded in training_data:
    training_source.append(input)
    training_target.append(target)
    training_target_encoded.append(target_encoded)
#     print(x.shape)
training_source = torch.cat(training_source,dim=0)[:,1:]
training_target = torch.cat(training_target,dim=0)
training_target_encoded = torch.cat(training_target_encoded,dim=0)

In [4]:
test_data = DataLoader(CustomDataset("Data/eval_data.csv"))
test_source = []
test_target = []
test_target_encoded = []
for input,target,test_encoded in test_data:
    test_source.append(input)
    test_target.append(target)
    test_target_encoded.append(test_encoded)
#     print(x.shape)
test_source = torch.cat(test_source,dim=0)[:,1:]
test_target = torch.cat(test_target,dim=0)
test_target_encoded = torch.cat(test_target_encoded,dim=0)

In [5]:
print(test_source.shape,test_target.shape,test_target_encoded.shape)
print(training_source.shape,training_target.shape,training_target_encoded.shape)

torch.Size([2000, 8]) torch.Size([2000, 9]) torch.Size([2000, 9, 27])
torch.Size([7000, 8]) torch.Size([7000, 9]) torch.Size([7000, 9, 27])


In [6]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers,bidirectional = 0):
        super(Encoder, self).__init__()
#         self.embedding = embedding
        self.embedding = nn.Embedding(26,input_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.encoder = nn.RNN(input_size, hidden_size,num_layers,batch_first = True,bidirectional=bool(self.bidirectional))
    
    def forward(self, input_):
        input_seq = self.embedding(input_)
        hidden = torch.zeros((self.bidirectional+1)*self.num_layers,input_seq.shape[0],self.hidden_size)
        _, encoder_hidden = self.encoder(input_seq,hidden)
        if(self.bidirectional == 1):
            encoder_hidden = torch.cat((encoder_hidden[0::2],encoder_hidden[1::2]),dim=2)
        return encoder_hidden
    
    def predict(self,input_str):
        input_sentence = torch.tensor([ord(c) - ord('a') for c in input_str])
        input_seq = self.embedding(input_sentence)
        hidden = torch.zeros((self.bidirectional+1)*self.num_layers,self.hidden_size)
        _, encoder_hidden = self.encoder(input_seq,hidden)
        if(self.bidirectional == 1):
            encoder_hidden = torch.cat((encoder_hidden[::2],encoder_hidden[1::2]),dim=1)
        return encoder_hidden
        
    
class Decoder(nn.Module):
    def __init__(self,input_size, hidden_size,num_layers,bidirectional = 0):
        super(Decoder, self).__init__()
#         self.embedding = embedding
        self.embedding = nn.Embedding(27,input_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.sequence_len = 9
        self.bidirectional = bidirectional
        
        self.dec_cells = nn.ModuleList([nn.RNNCell((bidirectional+1)*hidden_size+input_size, (bidirectional+1)*hidden_size)])
            
        self.decoder = nn.GRU((bidirectional+1)*hidden_size+input_size, (bidirectional+1)*hidden_size,batch_first = True)
        self.linear = nn.Linear((bidirectional+1)*hidden_size,26)
        self.output_layer = nn.LogSoftmax(dim = 2)
        self.output_layer_timestep = nn.LogSoftmax(dim = 1)
    
    def forward(self, context, target_,teacher_ratio):
        target_seq = self.embedding(target_)
        initial_hidden = torch.zeros_like(context)
        outputs = []
        
        hidden_states = []
        for timestep in range(self.sequence_len):
            h_t = []
            if(timestep == 0):
                h_t = self.dec_cells[0](torch.cat((target_seq[:,timestep],context[0]),dim=1), initial_hidden[0])
            else:
                input = []
                if(torch.rand(1).item() < teacher_ratio):
                    input = target_seq[:,timestep]
                else:
                    input = self.embedding(torch.argmax(outputs[-1],dim=1))
#                     print(torch.argmax(outputs[-1],dim=1)[0],target_[0,layer_idx])
                    
                h_t = self.dec_cells[0](torch.cat((input,context[0]),dim=1), hidden_states[-1])
            hidden_states.append(h_t)
            out = self.output_layer_timestep(self.linear(h_t))
            outputs.append(out)
    
 
        hidden_states = torch.cat(hidden_states,dim = 1).reshape(-1,9,context.shape[2])
        output_prob = torch.cat(outputs,dim = 1).reshape(-1,9,26)
        
        
#         out,_ = self.decoder(target_seq,context)
#         decoder_output = self.linear(hidden_states)
#         output_prob = self.output_layer(decoder_output)
        return output_prob

    def predict(self,context):
        target_seq = self.embedding(torch.tensor(1))
        initial_hidden = torch.zeros_like(context)
        outputs = []
        
        hidden_states = []
        for layer_idx in range(self.sequence_len):
            h_t = []
            if(layer_idx == 0):
                h_t = self.dec_cells[0](torch.cat((target_seq,context[0]),dim=0), initial_hidden[0])
            else:
                input = self.embedding(torch.argmax(outputs[-1],dim=0))
                    
                h_t = self.dec_cells[0](torch.cat((input,context[0]),dim=0), hidden_states[-1])
            hidden_states.append(h_t)
            out = nn.LogSoftmax(dim=0)(self.linear(h_t))
            outputs.append(out)
        
        output = ''.join([chr(torch.argmax(out,dim=0).item()+ord('a')) for out in outputs[:-1]])
        
        return output
        
        
        


In [7]:
input_size = 10
encoder = Encoder(input_size,32,1,1)
decoder = Decoder(input_size,32,1,1)
criterion = nn.KLDivLoss(reduction = "batchmean")
# optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()),lr = 0.001)
optimizer_encoder = optim.Adam(encoder.parameters(), lr=0.01)
# scheduler_encoder = optim.lr_scheduler.StepLR(optimizer_encoder, step_size=1000, gamma=0.5)
optimizer_decoder = optim.Adam(decoder.parameters(), lr=0.01)
# scheduler_decoder = optim.lr_scheduler.StepLR(optimizer_decoder, step_size=1000, gamma=0.5)
# optimizer_encoder = optim.SGD(encoder.parameters(), lr=0.2,momentum=0.9)
# optimizer_decoder = optim.SGD(decoder.parameters(), lr=0.2,momentum=0.9)

# Set up early stopping parameters
patience = 500  # Number of epochs to wait for improvement
best_val_loss = float('inf')
epochs_since_improvement = 0
loss_val = 0.0

num_epochs = 2000
for epoch in range(num_epochs):
    # Training Loop
    encoder.train()
    decoder.train()
    optimizer_encoder.zero_grad()
    optimizer_decoder.zero_grad()
#     optimizer.zero_grad()
    context = encoder(training_source)
    output = decoder(context,training_target,0.2)
    loss = criterion(output[:,:-1,:], training_target_encoded[:,1:,:-1])
    loss.backward()
    optimizer_encoder.step()
    optimizer_decoder.step()
#     scheduler_encoder.step()
#     scheduler_decoder.step()
#     optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}')
    
    # Validation Loop
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        context_ = encoder(test_source)
        output_ = decoder(context_,test_target,0.0)
        loss_val = criterion(output_[:,:-1,:], test_target_encoded[:,1:,:-1])
        print(f'Epoch [{epoch+1}/{num_epochs}], Eval Loss: {loss_val.item():.4f}')
    
        if loss_val < best_val_loss:
            best_val_loss = loss_val
            epochs_since_improvement = 0
            torch.save(encoder.state_dict(), 'Model/encoder.pth')
            torch.save(decoder.state_dict(), 'Model/decoder.pth')

        else:
            epochs_since_improvement += 1

    # Check if we should stop training early
    if epochs_since_improvement >= patience:
        print(f"Early stopping after {epoch+1} epochs with no improvement.")
        break

if loss_val < best_val_loss:
    torch.save(encoder.state_dict(), 'Model/encoder.pth')
    torch.save(decoder.state_dict(), 'Model/decoder.pth')

Epoch [1/2000], Train Loss: 26.2139
Epoch [1/2000], Eval Loss: 26.1419
Epoch [2/2000], Train Loss: 26.1134
Epoch [2/2000], Eval Loss: 26.0259
Epoch [3/2000], Train Loss: 26.0080
Epoch [3/2000], Eval Loss: 25.9625
Epoch [4/2000], Train Loss: 25.9366
Epoch [4/2000], Eval Loss: 25.9185
Epoch [5/2000], Train Loss: 25.8624
Epoch [5/2000], Eval Loss: 25.8373
Epoch [6/2000], Train Loss: 25.7672
Epoch [6/2000], Eval Loss: 25.7665
Epoch [7/2000], Train Loss: 25.6858
Epoch [7/2000], Eval Loss: 25.6920
Epoch [8/2000], Train Loss: 25.5817
Epoch [8/2000], Eval Loss: 25.6040
Epoch [9/2000], Train Loss: 25.4784
Epoch [9/2000], Eval Loss: 25.5027
Epoch [10/2000], Train Loss: 25.3745
Epoch [10/2000], Eval Loss: 25.3960
Epoch [11/2000], Train Loss: 25.2512
Epoch [11/2000], Eval Loss: 25.2863
Epoch [12/2000], Train Loss: 25.1588
Epoch [12/2000], Eval Loss: 25.1615
Epoch [13/2000], Train Loss: 25.0402
Epoch [13/2000], Eval Loss: 25.0386
Epoch [14/2000], Train Loss: 24.9619
Epoch [14/2000], Eval Loss: 25.0

Epoch [116/2000], Train Loss: 20.3348
Epoch [116/2000], Eval Loss: 21.3016
Epoch [117/2000], Train Loss: 20.3026
Epoch [117/2000], Eval Loss: 21.2446
Epoch [118/2000], Train Loss: 20.2919
Epoch [118/2000], Eval Loss: 21.1880
Epoch [119/2000], Train Loss: 20.2389
Epoch [119/2000], Eval Loss: 21.1784
Epoch [120/2000], Train Loss: 20.2253
Epoch [120/2000], Eval Loss: 21.1421
Epoch [121/2000], Train Loss: 20.1714
Epoch [121/2000], Eval Loss: 21.1440
Epoch [122/2000], Train Loss: 20.1712
Epoch [122/2000], Eval Loss: 21.1053
Epoch [123/2000], Train Loss: 20.1094
Epoch [123/2000], Eval Loss: 21.0925
Epoch [124/2000], Train Loss: 20.1155
Epoch [124/2000], Eval Loss: 21.0895
Epoch [125/2000], Train Loss: 20.0921
Epoch [125/2000], Eval Loss: 21.0740
Epoch [126/2000], Train Loss: 20.0639
Epoch [126/2000], Eval Loss: 20.9672
Epoch [127/2000], Train Loss: 19.9566
Epoch [127/2000], Eval Loss: 20.9556
Epoch [128/2000], Train Loss: 19.9307
Epoch [128/2000], Eval Loss: 20.9164
Epoch [129/2000], Train L

Epoch [227/2000], Train Loss: 13.0775
Epoch [227/2000], Eval Loss: 13.6897
Epoch [228/2000], Train Loss: 12.9835
Epoch [228/2000], Eval Loss: 13.6288
Epoch [229/2000], Train Loss: 12.8732
Epoch [229/2000], Eval Loss: 14.1053
Epoch [230/2000], Train Loss: 13.4138
Epoch [230/2000], Eval Loss: 15.4586
Epoch [231/2000], Train Loss: 14.8838
Epoch [231/2000], Eval Loss: 15.1764
Epoch [232/2000], Train Loss: 14.5553
Epoch [232/2000], Eval Loss: 15.2880
Epoch [233/2000], Train Loss: 14.6108
Epoch [233/2000], Eval Loss: 14.7387
Epoch [234/2000], Train Loss: 14.0967
Epoch [234/2000], Eval Loss: 14.5186
Epoch [235/2000], Train Loss: 13.8092
Epoch [235/2000], Eval Loss: 14.4948
Epoch [236/2000], Train Loss: 13.8468
Epoch [236/2000], Eval Loss: 14.3111
Epoch [237/2000], Train Loss: 13.6481
Epoch [237/2000], Eval Loss: 14.0845
Epoch [238/2000], Train Loss: 13.4052
Epoch [238/2000], Eval Loss: 13.9152
Epoch [239/2000], Train Loss: 13.2326
Epoch [239/2000], Eval Loss: 13.5892
Epoch [240/2000], Train L

Epoch [341/2000], Train Loss: 7.0427
Epoch [341/2000], Eval Loss: 7.2373
Epoch [342/2000], Train Loss: 6.9291
Epoch [342/2000], Eval Loss: 7.2047
Epoch [343/2000], Train Loss: 6.8590
Epoch [343/2000], Eval Loss: 7.1857
Epoch [344/2000], Train Loss: 6.8066
Epoch [344/2000], Eval Loss: 7.0857
Epoch [345/2000], Train Loss: 6.7154
Epoch [345/2000], Eval Loss: 6.9683
Epoch [346/2000], Train Loss: 6.6326
Epoch [346/2000], Eval Loss: 6.8830
Epoch [347/2000], Train Loss: 6.5764
Epoch [347/2000], Eval Loss: 6.8205
Epoch [348/2000], Train Loss: 6.5210
Epoch [348/2000], Eval Loss: 6.7707
Epoch [349/2000], Train Loss: 6.4658
Epoch [349/2000], Eval Loss: 6.7307
Epoch [350/2000], Train Loss: 6.4247
Epoch [350/2000], Eval Loss: 6.6852
Epoch [351/2000], Train Loss: 6.3757
Epoch [351/2000], Eval Loss: 6.6310
Epoch [352/2000], Train Loss: 6.3318
Epoch [352/2000], Eval Loss: 6.5778
Epoch [353/2000], Train Loss: 6.2973
Epoch [353/2000], Eval Loss: 6.5416
Epoch [354/2000], Train Loss: 6.2671
Epoch [354/200

Epoch [456/2000], Train Loss: 5.5832
Epoch [456/2000], Eval Loss: 5.9274
Epoch [457/2000], Train Loss: 5.5825
Epoch [457/2000], Eval Loss: 5.9269
Epoch [458/2000], Train Loss: 5.5804
Epoch [458/2000], Eval Loss: 5.9269
Epoch [459/2000], Train Loss: 5.5776
Epoch [459/2000], Eval Loss: 5.9266
Epoch [460/2000], Train Loss: 5.5757
Epoch [460/2000], Eval Loss: 5.9254
Epoch [461/2000], Train Loss: 5.5740
Epoch [461/2000], Eval Loss: 5.9259
Epoch [462/2000], Train Loss: 5.5734
Epoch [462/2000], Eval Loss: 5.9264
Epoch [463/2000], Train Loss: 5.5707
Epoch [463/2000], Eval Loss: 5.9259
Epoch [464/2000], Train Loss: 5.5713
Epoch [464/2000], Eval Loss: 5.9252
Epoch [465/2000], Train Loss: 5.5705
Epoch [465/2000], Eval Loss: 5.9248
Epoch [466/2000], Train Loss: 5.5654
Epoch [466/2000], Eval Loss: 5.9242
Epoch [467/2000], Train Loss: 5.5643
Epoch [467/2000], Eval Loss: 5.9233
Epoch [468/2000], Train Loss: 5.5624
Epoch [468/2000], Eval Loss: 5.9225
Epoch [469/2000], Train Loss: 5.5618
Epoch [469/200

Epoch [569/2000], Train Loss: 23.2606
Epoch [569/2000], Eval Loss: 23.1788
Epoch [570/2000], Train Loss: 22.7600
Epoch [570/2000], Eval Loss: 22.8126
Epoch [571/2000], Train Loss: 22.3809
Epoch [571/2000], Eval Loss: 22.5660
Epoch [572/2000], Train Loss: 22.1056
Epoch [572/2000], Eval Loss: 22.2652
Epoch [573/2000], Train Loss: 21.7594
Epoch [573/2000], Eval Loss: 21.9120
Epoch [574/2000], Train Loss: 21.4315
Epoch [574/2000], Eval Loss: 21.6163
Epoch [575/2000], Train Loss: 21.1343
Epoch [575/2000], Eval Loss: 21.3711
Epoch [576/2000], Train Loss: 20.8653
Epoch [576/2000], Eval Loss: 21.1462
Epoch [577/2000], Train Loss: 20.6032
Epoch [577/2000], Eval Loss: 20.9498
Epoch [578/2000], Train Loss: 20.3437
Epoch [578/2000], Eval Loss: 20.6724
Epoch [579/2000], Train Loss: 20.0350
Epoch [579/2000], Eval Loss: 20.4484
Epoch [580/2000], Train Loss: 19.7560
Epoch [580/2000], Eval Loss: 20.2401
Epoch [581/2000], Train Loss: 19.5349
Epoch [581/2000], Eval Loss: 19.9637
Epoch [582/2000], Train L

Epoch [679/2000], Train Loss: 9.7946
Epoch [679/2000], Eval Loss: 10.2658
Epoch [680/2000], Train Loss: 9.7199
Epoch [680/2000], Eval Loss: 10.1965
Epoch [681/2000], Train Loss: 9.6834
Epoch [681/2000], Eval Loss: 10.2652
Epoch [682/2000], Train Loss: 9.7558
Epoch [682/2000], Eval Loss: 10.4901
Epoch [683/2000], Train Loss: 10.0123
Epoch [683/2000], Eval Loss: 11.1316
Epoch [684/2000], Train Loss: 10.6581
Epoch [684/2000], Eval Loss: 11.6730
Epoch [685/2000], Train Loss: 11.2240
Epoch [685/2000], Eval Loss: 11.2525
Epoch [686/2000], Train Loss: 10.7120
Epoch [686/2000], Eval Loss: 10.8855
Epoch [687/2000], Train Loss: 10.4018
Epoch [687/2000], Eval Loss: 10.6915
Epoch [688/2000], Train Loss: 10.2947
Epoch [688/2000], Eval Loss: 10.2538
Epoch [689/2000], Train Loss: 9.7987
Epoch [689/2000], Eval Loss: 10.5402
Epoch [690/2000], Train Loss: 10.1061
Epoch [690/2000], Eval Loss: 10.1941
Epoch [691/2000], Train Loss: 9.7588
Epoch [691/2000], Eval Loss: 10.2300
Epoch [692/2000], Train Loss: 9

Epoch [792/2000], Eval Loss: 10.4880
Epoch [793/2000], Train Loss: 10.1090
Epoch [793/2000], Eval Loss: 10.2787
Epoch [794/2000], Train Loss: 9.8936
Epoch [794/2000], Eval Loss: 10.1891
Epoch [795/2000], Train Loss: 9.7786
Epoch [795/2000], Eval Loss: 9.9592
Epoch [796/2000], Train Loss: 9.5553
Epoch [796/2000], Eval Loss: 9.8087
Epoch [797/2000], Train Loss: 9.4309
Epoch [797/2000], Eval Loss: 9.6968
Epoch [798/2000], Train Loss: 9.2890
Epoch [798/2000], Eval Loss: 9.5531
Epoch [799/2000], Train Loss: 9.1609
Epoch [799/2000], Eval Loss: 9.3924
Epoch [800/2000], Train Loss: 9.0202
Epoch [800/2000], Eval Loss: 9.2400
Epoch [801/2000], Train Loss: 8.8474
Epoch [801/2000], Eval Loss: 9.1082
Epoch [802/2000], Train Loss: 8.7213
Epoch [802/2000], Eval Loss: 9.0217
Epoch [803/2000], Train Loss: 8.6321
Epoch [803/2000], Eval Loss: 8.8678
Epoch [804/2000], Train Loss: 8.4902
Epoch [804/2000], Eval Loss: 8.8004
Epoch [805/2000], Train Loss: 8.4401
Epoch [805/2000], Eval Loss: 8.6737
Epoch [806/

Epoch [906/2000], Eval Loss: 6.6942
Epoch [907/2000], Train Loss: 6.4074
Epoch [907/2000], Eval Loss: 6.6819
Epoch [908/2000], Train Loss: 6.3957
Epoch [908/2000], Eval Loss: 6.6651
Epoch [909/2000], Train Loss: 6.3785
Epoch [909/2000], Eval Loss: 6.6462
Epoch [910/2000], Train Loss: 6.3662
Epoch [910/2000], Eval Loss: 6.6337
Epoch [911/2000], Train Loss: 6.3540
Epoch [911/2000], Eval Loss: 6.6189
Epoch [912/2000], Train Loss: 6.3394
Epoch [912/2000], Eval Loss: 6.6078
Epoch [913/2000], Train Loss: 6.3295
Epoch [913/2000], Eval Loss: 6.6004
Epoch [914/2000], Train Loss: 6.3203
Epoch [914/2000], Eval Loss: 6.5888
Epoch [915/2000], Train Loss: 6.3092
Epoch [915/2000], Eval Loss: 6.5769
Epoch [916/2000], Train Loss: 6.2965
Epoch [916/2000], Eval Loss: 6.5665
Epoch [917/2000], Train Loss: 6.2865
Epoch [917/2000], Eval Loss: 6.5555
Epoch [918/2000], Train Loss: 6.2766
Epoch [918/2000], Eval Loss: 6.5426
Epoch [919/2000], Train Loss: 6.2670
Epoch [919/2000], Eval Loss: 6.5350
Epoch [920/2000

In [9]:
encoder = Encoder(input_size,32,1,1)
decoder = Decoder(input_size,32,1,1)
encoder.load_state_dict(torch.load('Model/encoder.pth'))  
decoder.load_state_dict(torch.load('Model/decoder.pth')) 
encoder.eval()
decoder.eval()
with torch.no_grad():
    context_ = encoder(test_source)
    output_ = decoder(context_,test_target,0.0)
    loss_val = criterion(output_[:,:-1,:], test_target_encoded[:,1:,:-1])
    print(f'Epoch [{epoch+1}/{num_epochs}], Eval Loss: {loss_val.item():.4f}')


Epoch [972/2000], Eval Loss: 5.9203


In [10]:
encoder.eval()
decoder.eval()
with torch.no_grad():
    context = encoder(test_source)
    output = decoder(context,test_target,0.0)
    actual = torch.argmax(test_target_encoded[:,1:,:-1],dim=2)
    predictions = torch.argmax(output[:,:-1,:],dim=2)
    wrong_pred = torch.where(predictions != actual,1.0,0.0)
    print(f'Average(over batch) no. of Wrong predictions per sequence : {torch.sum(wrong_pred) / wrong_pred.shape[0]:.4f}')

Average(over batch) no. of Wrong predictions per sequence : 4.0140


In [11]:
# Function to check how many characters match in the two strings
def check(pred: str, true: str):
    correct = 0
    for a, b in zip(pred, true):
        if a == b:
            correct += 1

    # Prediction is more than 8 letters, so penalize for every extra letter.
    correct -= max(0, len(pred) - len(true))
    correct = max(0, correct)
    return correct

# Function to score the model's performance
def evaluate(encoder, decoder):

    # Train data
    print("Obtaining results for training data:")
    train_data = pd.read_csv("Data/train_data.csv").to_numpy()
    results = {
        "pred": [],
        "true": [],
        "score": [],
    }
    correct = [0 for _ in range(9)]
    for x, y in train_data:
        pred = decoder.predict(encoder.predict(x))
        score = check(pred, y)
        results["pred"].append(pred)
        results["true"].append(y)
        results["score"].append(score)

        correct[score] += 1
    print("Train dataset results:")
    for num_chr in range(9):
        print(
            f"Number of predictions with {num_chr} correct predictions: {correct[num_chr]}"
        )
    points = sum(correct[4:6]) * 0.5 + sum(correct[6:])
    print(f"Points: {points}")
    # Save predicitons and true sentences to inspect manually if required.
    pd.DataFrame.from_dict(results).to_csv("results_train.csv", index=False)

    #----------------------------------------------------------------------------------

    print("Obtaining metrics for eval data:")
    eval_data = pd.read_csv("Data/eval_data.csv").to_numpy()
    results = {
        "pred": [],
        "true": [],
        "score": [],
    }
    correct = [0 for _ in range(9)]
    for x, y in eval_data:
        pred = decoder.predict(encoder.predict(x))
        score = check(pred, y)
        results["pred"].append(pred)
        results["true"].append(y)
        results["score"].append(score)

        correct[score] += 1
    print("Eval dataset results:")
    for num_chr in range(9):
        print(
            f"Number of predictions with {num_chr} correct predictions: {correct[num_chr]}"
        )
    points = sum(correct[4:6]) * 0.5 + sum(correct[6:])
    marks = round(min(2, points / 1400 * 2) * 2) / 2  # Rounds to the nearest 0.5
    print(f"Points: {points}")
    print(f"Marks: {marks}")
    # Save predicitons and true sentences to inspect manually if required.
    pd.DataFrame.from_dict(results).to_csv("results_eval.csv", index=False)


In [12]:
evaluate(encoder,decoder)

Obtaining results for training data:
Train dataset results:
Number of predictions with 0 correct predictions: 51
Number of predictions with 1 correct predictions: 364
Number of predictions with 2 correct predictions: 990
Number of predictions with 3 correct predictions: 1831
Number of predictions with 4 correct predictions: 1854
Number of predictions with 5 correct predictions: 1240
Number of predictions with 6 correct predictions: 524
Number of predictions with 7 correct predictions: 135
Number of predictions with 8 correct predictions: 11
Points: 2217.0
Obtaining metrics for eval data:
Eval dataset results:
Number of predictions with 0 correct predictions: 28
Number of predictions with 1 correct predictions: 146
Number of predictions with 2 correct predictions: 367
Number of predictions with 3 correct predictions: 549
Number of predictions with 4 correct predictions: 524
Number of predictions with 5 correct predictions: 265
Number of predictions with 6 correct predictions: 106
Number